### Random traveling

This is a simple spatial agent-based model that models agents with different probabilities of random movement. Two sub-populations are created, one with a low probability of traveling and one with higher probability. Each agent moves randomly to a neighboring cell in the grid, and as such performs a random walk.

In [ ]:
import random
import numpy as np

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid

from bokeh.models import ColumnDataSource, BasicTicker, CategoricalTicker, LinearColorMapper
from bokeh.plotting import output_notebook, figure, show
from bokeh.layouts import row
from bokeh.transform import transform
from bokeh.io import push_notebook

In [ ]:
#embed figures in the notebook
output_notebook()

#### Create the model

In [ ]:
class Traveler(Agent):
    def __init__(self, unique_id, model, travel_prob):
        super().__init__(unique_id, model)
        self.travel_prob = travel_prob
    
    def _travel(self):
        possible_steps = self.model.grid.get_neighborhood(
                self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
    
    def move(self):
        if np.random.binomial(1,self.travel_prob):
            self._travel()
    
    def step(self):
        self.move()

In [ ]:
class TravelModel(Model):
    def __init__(self, N, travel_probs, width, height):
        self.number_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        # Create agents
        for i in range(self.number_agents):
            # Give randomly some agents high or 
            # low probability to travel
            a = Traveler(
                i, self, travel_probs[int(np.random.binomial(1,0.5))])
            self.schedule.add(a)
        #put agents in initial position    
        self._init_travelers()
    
    def step(self):
        self.schedule.step()
            
    def _init_travelers(self):
        #agents are initially randomly distributed in the center of grid
        initial_dim = np.ceil(np.sqrt(self.number_agents))
        
        for agent in self.schedule.agents:
            center_x = self.grid.width/2
            center_y = self.grid.height/2
        
            x = np.random.randint(center_x - initial_dim, center_x + initial_dim)
            y = np.random.randint(center_y - initial_dim, center_y + initial_dim)
            self.grid.place_agent(agent, (x, y))

#### Visualization

In [ ]:
def update_agents_data(agents):
    new_data = {}
    agents_positions = np.array([agent.pos for agent in agents])
    new_data['x'] = np.array([pos[0] for pos in agents_positions])
    new_data['y'] = np.array([pos[1] for pos in agents_positions])
    new_data['travel_prob'] = np.array([agent.travel_prob for agent in agents])
    return new_data

In [ ]:
def show_data(agents_data, travel_probs, grid):
    colors = ['blue', 'red']
    mapper = LinearColorMapper(palette=colors, 
                               low=travel_probs[0], 
                               high=travel_probs[1])

    p = figure(plot_width=500, plot_height=500, title='Travel Model',
               x_range=[str(val) for val in range(grid.width)], 
               y_range=[str(val) for val in range(grid.height)],
               x_axis_location='below')

    p.circle(x='x', y='y', size=10, source=agents_data,
       line_color=None, fill_color=transform('travel_prob', mapper))

    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = '0pt'
    
    show(p, notebook_handle=True)

#### Run the model

In [ ]:
number_agents = 50
grid_height = 50
grid_width = 50
travel_probs = (1e-4, 0.01)

In [ ]:
model = TravelModel(number_agents, travel_probs, grid_width, grid_height)

In [ ]:
agents_data = ColumnDataSource(update_agents_data(model.schedule.agents))
show_data(agents_data, travel_probs, model.grid)

number_steps = 10000
log_step = number_steps / 100
for i in range(number_steps):
    model.step()
    if not (i % log_step):
        agents_data.data = update_agents_data(model.schedule.agents)
        push_notebook() 

### Visualization using Mesa

This doesn't seem to work.

In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

In [ ]:
def agent_portrayal(agent):
    color = "red" if agent.long_dist_prob >= 0.5 \
                  else "blue" 
    return {"Shape": "circle",
            "Color": "red",
            "Filled": "true",
            "Layer": 0,
            "r": 0.5}

grid = CanvasGrid(agent_portrayal, grid_width, grid_height, 1000, 1000)

server = ModularServer(TravelModel,
                       [grid],
                       "TravelModel",
                       {"N": number_agents, 
                        "long_dist_prob": long_dist_probs, 
                        "width": grid_width, 
                        "height": grid_height})

In [ ]:
server.port = 8521 # The default
server.launch()